In [2]:
%%configure 
{
    "script_location": "s3://shivam-trial-s3/code_artefacts/glue/scripts/",
    "--TempDir": "s3://shivam-trial-s3/code_artefacts/glue/temp/"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
The following configurations have been updated: {'script_location': 's3://shivam-trial-s3/code_artefacts/glue/scripts/', '--TempDir': 's3://shivam-trial-s3/code_artefacts/glue/temp/'}


In [1]:
%idle_timeout 15
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Current idle_timeout is None minutes.
idle_timeout has been set to 15 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 15
Session ID: 33f55cf5-0db2-4580-be4c-dc1b060a959b
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session 33f55cf5-0db2-4580-be4c-dc1b060a959b to get into ready status...
Session 33f55cf5-0db2-4580-be4c-dc1b060a959b has be

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
user_df = spark.sql("SELECT * FROM stage.stage_user")
content_df = spark.sql("SELECT * FROM stage.stage_content")
event_df = spark.sql("SELECT * FROM stage.stage_event")

user_segmentation = spark.sql("SELECT * FROM mart.mart_user_segmentation")

In [19]:
a = jan_users_segmented.filter(jan_users_segmented.user_segment.isNull())
a.show(truncate=False)

+--------+----------+------------+
|deviceid|install_dt|user_segment|
+--------+----------+------------+
+--------+----------+------------+


In [18]:
# Users installing app on January 2024
jan_users = user_df.filter((user_df.install_dt >= "2024-01-01") & (user_df.install_dt < "2024-02-01")).select("deviceid", "install_dt")

# User Segmentation for January Users
jan_users_segmented = jan_users.join(user_segmentation.select("deviceid","user_segment"),"deviceid","inner")

# All events related to January 2024 devices
jan_events = event_df.join(jan_users_segmented,"deviceid","inner")

# Convert eventtimestamp to date column for Day wise metrics
event_with_date = jan_events.withColumn("event_date", to_date(col("eventtimestamp")))

# Date difference for events from installed date
event_with_diff = event_with_date.withColumn("days_after_install", datediff(event_with_date.event_date, event_with_date.install_dt))

# Events for D0, D1, D3 and D7 metrics
filtered_days = event_with_diff.filter(event_with_diff.days_after_install.isin(0, 1, 3, 7)).select("deviceid", "user_segment", "days_after_install").distinct()

# D0 users corresponding to all users who installed in Jan 2024. Adding them in case they never returned to app after installing it.
d0_users = jan_users_segmented.select("deviceid", "user_segment").distinct()

# Join D0 users with filtered_days for D1, D3, D7 and group by user_segment
retention_by_segment = d0_users.join(filtered_days, ["deviceid", "user_segment"], "inner") \
    .groupBy("user_segment").agg(
        countDistinct("deviceid").alias("D0"),
        countDistinct(when(col("days_after_install") == 1, col("deviceid"))).alias("D1"),
        countDistinct(when(col("days_after_install") == 3, col("deviceid"))).alias("D3"),
        countDistinct(when(col("days_after_install") == 7, col("deviceid"))).alias("D7")
    )

# Calculate the rates & keep in order
retention_rates = retention_by_segment.withColumns({
    "D0_rate": lit(100.0),
    "D1_rate": round((col("D1") / col("D0")) * 100, 2),
    "D3_rate": round((col("D3") / col("D0")) * 100, 2),
    "D7_rate": round((col("D7") / col("D0")) * 100, 2)}).orderBy("user_segment")


In [20]:
retention_rates.show(truncate=False)

+--------------------------+-----+-----+-----+-----+-------+-------+-------+-------+
|user_segment              |D0   |D1   |D3   |D7   |D0_rate|D1_rate|D3_rate|D7_rate|
+--------------------------+-----+-----+-----+-----+-------+-------+-------+-------+
|Content Readers - English |29   |28   |29   |27   |100.0  |96.55  |100.0  |93.1   |
|Content Readers - Hindi   |4    |4    |4    |4    |100.0  |100.0  |100.0  |100.0  |
|Loyal Users - English     |14321|13694|13561|13538|100.0  |95.62  |94.69  |94.53  |
|Loyal Users - Hindi       |2359 |2299 |2288 |2263 |100.0  |97.46  |96.99  |95.93  |
|Notification Engagers     |1    |1    |1    |1    |100.0  |100.0  |100.0  |100.0  |
|Occasional Users - English|17020|7262 |4228 |2398 |100.0  |42.67  |24.84  |14.09  |
|Occasional Users - Hindi  |3422 |1514 |828  |431  |100.0  |44.24  |24.2   |12.59  |
|Other                     |4344 |3796 |3486 |3189 |100.0  |87.38  |80.25  |73.41  |
|Social Sharers - English  |6    |6    |5    |6    |100.0  |100.0